In [27]:
from census import Census
from config import (census_key)
from us import states
import pandas as pd

c = Census(census_key, year=2013)

In [28]:
#c.sf1.state_county_tract('NAME', states.TX.fips, '113', Census.ALL)

In [29]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E", "B23025_005E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Count"]]

# Visualize
print(len(census_pd))
census_pd.head()

33120


,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Count
0,08518,5217.0,41.5,74286.0,33963.0,170.0,3.258578,321.0
1,08520,27468.0,37.4,90293.0,37175.0,1834.0,6.676860,1418.0
2,08525,4782.0,47.1,118656.0,59848.0,43.0,0.899205,138.0
3,08527,54867.0,42.2,88588.0,37021.0,2191.0,3.993293,2910.0
4,08528,245.0,48.5,58676.0,49117.0,0.0,0.000000,0.0


In [30]:
dallasZips = pd.read_csv("../Resources/DallasZipCodes.csv")
dallasZips.head()

,City,Population,ZipCode
0,Addison,"12,414",75001
1,Carrollton,"46,364",75006
2,Carrollton,0,75011
3,Irving,0,75014
4,Irving,0,75015


In [31]:
print(census_pd.dtypes)
print(dallasZips.dtypes)

Zipcode                object
Population            float64
Median Age            float64
Household Income      float64
Per Capita Income     float64
Poverty Count         float64
Poverty Rate          float64
Unemployment Count    float64
dtype: object
City          object
Population    object
ZipCode        int64
dtype: object


In [32]:
census_pd["Zipcode"] = census_pd["Zipcode"].astype(int)
mask = [zipcode in dallasZips.ZipCode.values for zipcode in census_pd.Zipcode]
dallasSub = census_pd[mask]

In [33]:
dallasSub.head()

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Count
25853,75001,13214.0,32.2,57633.0,43919.0,1722.0,13.031633,433.0
25855,75006,47590.0,34.8,56985.0,27010.0,6927.0,14.555579,1996.0
25860,75019,39551.0,37.9,110445.0,48040.0,1624.0,4.106091,985.0
25871,75038,28161.0,30.0,48130.0,28391.0,4581.0,16.267178,1512.0
25872,75039,12197.0,32.2,73503.0,65135.0,819.0,6.714766,264.0


In [34]:
dallasSubMerge = pd.merge(dallasSub,dallasZips[['ZipCode','City']],left_on='Zipcode', right_on='ZipCode', how='inner')
dallasSubMerge = dallasSubMerge.drop(columns=["ZipCode"])
dallasSubMerge

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Count,City
0,75001,13214.0,32.2,57633.0,43919.0,1722.0,13.031633,433.0,Addison
1,75006,47590.0,34.8,56985.0,27010.0,6927.0,14.555579,1996.0,Carrollton
2,75019,39551.0,37.9,110445.0,48040.0,1624.0,4.106091,985.0,Coppell
3,75038,28161.0,30.0,48130.0,28391.0,4581.0,16.267178,1512.0,Irving
4,75039,12197.0,32.2,73503.0,65135.0,819.0,6.714766,264.0,Irving
5,75040,59792.0,32.4,54664.0,20387.0,9346.0,15.630854,3054.0,Garland
6,75041,30684.0,30.9,40136.0,16646.0,6975.0,22.731717,1659.0,Garland
7,75042,40185.0,30.0,39811.0,15085.0,10577.0,26.320766,2490.0,Garland
8,75043,58120.0,34.0,55149.0,24046.0,6512.0,11.204405,3233.0,Garland
9,75044,41578.0,38.9,64302.0,30021.0,3740.0,8.995142,2026.0,Garland


In [67]:
#drop NaN values
DroppedData = dallasSubMerge.dropna(how="any")


DroppedData.describe()

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Count
count,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000
mean,75163.107143,29725.595238,33.839286,54701.773810,29463.619048,5495.119048,18.569866,1385.226190
std,78.922567,19023.184205,4.844995,23213.807165,17053.465643,5054.688351,10.885028,1051.667149
min,75001.000000,546.000000,27.200000,15258.000000,9484.000000,120.000000,1.255099,29.000000
25%,75081.750000,15463.500000,30.850000,37421.250000,17514.750000,1823.250000,10.446745,576.000000
50%,75204.500000,26806.500000,32.700000,50161.000000,24275.000000,4051.000000,16.043116,1114.500000
75%,75228.250000,40825.250000,36.100000,67763.000000,35511.250000,7877.750000,26.047772,1984.750000
max,75254.000000,89743.000000,56.500000,147153.000000,91866.000000,26920.000000,53.113553,4848.000000


In [22]:
dallasSubMerge.to_csv("../Resources/DallasCensusData.csv", index=False)